## CS 585: Natural Langugae Processing 

## HMM for POS tagging

In [1]:
from collections import Counter, defaultdict
import math
import numpy as np
import os.path
import urllib.request
import nltk

## Download and read training data

In [2]:
def download_data():
    url = 'https://www.dropbox.com/s/ty7cclxiob3ajog/data.txt?dl=1'
    urllib.request.urlretrieve(url, 'data.txt')

In [3]:
def read_labeled_data(filename):

    """
        Read Training Data
    """

    file = open(filename)
    Text = file.read()
    text = Text.strip('\n').split('\n')
    sentences = []
    tags = []
    sentence = []
    tag = []
    for text1 in text:
        l = text1.split(' ')
        if (len(l) == 2):
            sentence.append(l[0])
            tag.append(l[1])
        else:
            sentences.append(sentence)
            tags.append(tag)
            sentence = []
            tag = []
    return sentences,tags

## HMM Implementation

In [4]:
class HMM:
    def __init__(self,smoothing):
        self.smoothing = smoothing

In [5]:
def iter_bigrams(l=[]):
        return (l[i:i + 2] for i in range(len(l) - 1))
HMM.iter_bigrams = iter_bigrams

In [6]:
def fit_transition_probas(self, tags):
    '''
    Compute the transition probabilities.
    '''
    self.transition_probas = defaultdict(lambda: Counter())
    for tag in tags:
        for bigram in iter_bigrams(tag):
            self.transition_probas[bigram[0]].update([bigram[-1]])
    unique_tags = set(x for l in tags for x in l)
    
    self.states = list(unique_tags)
    
    for tag,tag_counts in self.transition_probas.items():
        total = sum(tag_counts.values())
        for i in self.transition_probas.values():
            for j in unique_tags:
                if(j not in i):
                    i.update({j:0})
        self.transition_probas[tag] = {word: count/total for word,count in tag_counts.items()}
HMM.fit_transition_probas = fit_transition_probas

In [7]:
def fit_emission_probas(self, sentences, tags):
    '''
    Compute the emission probabilities
    '''
    self.emission_probas = defaultdict(lambda: Counter())
    for tag,sentence in zip(tags,sentences):
        for t,s in zip(tag,sentence):
            for bigram in iter_bigrams([t,s]):
                self.emission_probas[bigram[0]].update([bigram[-1]])
    
    unique_words = set(x for l in self.emission_probas.values() for x in l)
    
    for tag,word_counts in self.emission_probas.items():
        total = sum(word_counts.values())
        for i in self.emission_probas.values():
            for j in unique_words:
                if (j not in i):
                    i.update({j:0})
        self.emission_probas[tag] = {w: count/total for w,count in word_counts.items()}
                
HMM.fit_emission_probas = fit_emission_probas

In [8]:
def fit_start_probas(self, tags):
    '''
    Compute the start state probs
    '''
    counts = Counter(tag[0] for tag in tags)
    total = sum(counts.values())
    self.start_probas = {w: counts[w]/total for w in self.states}
HMM.fit_start_probas = fit_start_probas

In [9]:
def fit(self,sentences,tags):
    '''
    Fit the model to the training data
    '''
    self.fit_transition_probas(tags)
    self.fit_emission_probas(sentences,tags)
    self.fit_start_probas(tags)
HMM.fit = fit

In [10]:
def viterbi(self,sentence):
    '''
    Compute Hidden POS sequence using viterbi algorithm
    '''

    states = self.states
    viterbi = np.zeros((len(states),len(sentence)))
    backp = np.zeros((len(states),len(sentence)),np.int)
    
    for i in range(len(states)):
        viterbi[i][0] = self.start_probas[states[i]] * self.emission_probas[states[i]][sentence[0]]
        backp[i][0] = 0
    
    for i in range(1,len(sentence)):
        for post_state in range(len(states)):
            v_max = []
            b_max = []
            for pre_state in range(len(states)):
                v_max.append(viterbi[pre_state][i-1]* self.transition_probas[states[pre_state]][states[post_state]] \
                            * self.emission_probas[states[post_state]][sentence[i]])
                b_max.append(viterbi[pre_state][i-1] * self.transition_probas[states[pre_state]][states[post_state]])
            viterbi[post_state][i] = max(v_max)
            backp[post_state][i] = np.argmax(b_max)
        
    end_state = np.argmax(viterbi[:,len(sentence)-1])
    cur_state = backp[end_state][len(sentence)-1]
    path = [end_state]
    i = len(sentence) - 2
    while i!=-1:
        path.insert(0,cur_state)
        cur_state = backp[cur_state][i]
        i -= 1
    
    return list(np.array(states)[path]),np.max(viterbi[:,len(sentence)-1])
HMM.viterbi = viterbi

In [11]:
if __name__ == '__main__':
    '''
    M
    '''
    fname = 'data.txt'
    if not os.path.isfile(fname):
        download_data()
    sentences, tags = read_labeled_data(fname)

    model = HMM(0.01)
    model.fit(sentences, tags)
    print('model has %d states' % len(model.states))
    print(model.states)
    sentence = nltk.word_tokenize("Look at what happened")
    print('predicted parts of speech for the sentence %s' % str(sentence))
    print(model.viterbi(sentence))

model has 34 states
['NNP', 'TO', '$', 'NNPS', 'IN', 'VBP', 'PRP$', 'EX', 'NN', 'JJ', '.', 'VBG', 'VBZ', 'PRP', 'MD', 'WRB', ':', ',', 'VBN', 'NNS', 'RB', 'JJR', 'JJS', 'CC', 'WDT', 'CD', 'WP', 'RBR', 'POS', 'DT', 'VBD', "''", 'VB', '``']
predicted parts of speech for the sentence ['Look', 'at', 'what', 'happened']
(['VB', 'IN', 'WP', 'VBD'], 3.3172212325675426e-10)


In [12]:
nltk.help.upenn_tagset()

$: dollar
    $ -$ --$ A$ C$ HK$ M$ NZ$ S$ U.S.$ US$
'': closing quotation mark
    ' ''
(: opening parenthesis
    ( [ {
): closing parenthesis
    ) ] }
,: comma
    ,
--: dash
    --
.: sentence terminator
    . ! ?
:: colon or ellipsis
    : ; ...
CC: conjunction, coordinating
    & 'n and both but either et for less minus neither nor or plus so
    therefore times v. versus vs. whether yet
CD: numeral, cardinal
    mid-1890 nine-thirty forty-two one-tenth ten million 0.5 one forty-
    seven 1987 twenty '79 zero two 78-degrees eighty-four IX '60s .025
    fifteen 271,124 dozen quintillion DM2,000 ...
DT: determiner
    all an another any both del each either every half la many much nary
    neither no some such that the them these this those
EX: existential there
    there
FW: foreign word
    gemeinschaft hund ich jeux habeas Haementeria Herr K'ang-si vous
    lutihaw alai je jour objets salutaris fille quibusdam pas trop Monte
    terram fiche oui corporis ...
IN: preposition or